In [157]:
import numpy as np 
import pandas as pd 
import warnings
warnings.filterwarnings("ignore")

from millify import millify

In [158]:
from pathlib import Path
p = Path().cwd()
print(f"original File path: {p}")
stringpath= str(p)[0:str(p).rfind('\\')] + '\\data'
print(f"Updated File Path to pick from Data Library: { stringpath}")
ngsimfile=stringpath + '/' + 'Next_Generation_Simulation__NGSIM__Vehicle_Trajectories_and_Supporting_Data.csv'
ngsim = pd.read_csv(ngsimfile,low_memory=False)
print(f"{ngsim.duplicated().sum()} duplicate values have been removed")
ngsim.drop_duplicates(inplace=True)

original File path: c:\Users\StudentAccount\Python\Capstone\gitcodelocation\DataDrivenCarFollowing\datadrivencarfollowing-v1\scripts
Updated File Path to pick from Data Library: c:\Users\StudentAccount\Python\Capstone\gitcodelocation\DataDrivenCarFollowing\datadrivencarfollowing-v1\data
704000 duplicate values have been removed


In [ ]:
ngsim['v_length']= ngsim['v_length']*0.3048
ngsim['Space_Headway']= ngsim['Space_Headway']*0.3048
ngsim['v_Vel']= ngsim['v_Vel']*0.3048
ngsim['v_Acc']= ngsim['v_Acc']*0.3048
ngsim['Relative_Time']= ngsim['Global_Time'] - ngsim['Global_Time'].min() + 1
ngsim= ngsim.drop(columns=['Movement', 'Direction','Section_ID','Int_ID','D_Zone','O_Zone','Lane_ID','Following','v_Width','Total_Frames','Frame_ID'])


In [ ]:
#filtered_ngsim= ngsim[((ngsim['Location'] == 'us-101') | (ngsim['Location'] == 'i-80')) & (ngsim['Preceding']>0) & (ngsim['Following']>0) ]
ngsim['Preceding_Vehicle_Class'] = np.NaN
ngsim['Rear_to_Front_Space_Headway'] = np.NaN
ngsim['Follower_Minus_Preceding_Velocity_diff'] = np.NaN

ngsim["v_Class_Name"] = ngsim["v_Class"].map({1:"Motorcycle", 2: "Car", 3: "Heavy Vehicle"})

filtered_ngsim_U= ngsim[((ngsim['Location'] == 'us-101')) & ((ngsim['Vehicle_ID'] <= 1000 )  )]
#These two vehicles have a bad trajectory and keep on following each other in a single lane in a matter of 
filtered_ngsim_I= ngsim[(ngsim['Location'] == 'i-80') & ( ((ngsim['Vehicle_ID'] != 1748 )  & (ngsim['Preceding'] != 1734 )) &(  (ngsim['Vehicle_ID'] != 1734 )  & (ngsim['Preceding'] != 1748 )))& ((ngsim['Vehicle_ID'] <= 1000 )  )]
#filtered_ngsim_I= ngsim[(ngsim['Location'] == 'i-80')]

In [ ]:
filtered_ngsim_U.set_index(['Vehicle_ID', 'Preceding','Relative_Time'], append=True,inplace=True)
print(filtered_ngsim_U.index.is_monotonic_increasing)
filtered_ngsim_U=filtered_ngsim_U.sort_index()
print(filtered_ngsim_U.index.is_monotonic_increasing)

True
True


In [114]:
filtered_ngsim_U[((filtered_ngsim_U['Space_Headway'] == 0 ) & (filtered_ngsim_U.index.get_level_values("Preceding") > 0 )) ]

,,,,Global_Time,Local_X,Local_Y,Global_X,Global_Y,v_length,v_Class,v_Vel,v_Acc,Space_Headway,Time_Headway,Location,Preceding_Vehicle_Class,Rear_to_Front_Space_Headway,Follower_Minus_Preceding_Velocity_diff,v_Class_Name
,Vehicle_ID,Preceding,Relative_Time,,,,,,,,,,,,,,,,
76361,99,92,1117684929401,1118847948500,12.942,1420.624,6452132.084,1872437.889,2.5908,1,17.373600,-0.064008,0.0,0.0,us-101,NaN,NaN,NaN,Motorcycle
107058,806,814,1117685210701,1118848229800,6.699,796.216,6451666.005,1872853.511,4.4196,2,2.795016,0.000000,0.0,0.0,us-101,NaN,NaN,NaN,Car
452347,366,370,1117685952801,1118848971900,12.613,835.132,6451695.487,1872819.732,2.1336,1,8.695944,1.816608,0.0,0.0,us-101,NaN,NaN,NaN,Motorcycle


In [115]:
filtered_ngsim_I[((filtered_ngsim_I['Space_Headway'] == 0 ) & (filtered_ngsim_I['Preceding'] > 0 )) ]

,Vehicle_ID,Global_Time,Local_X,Local_Y,Global_X,Global_Y,v_length,v_Class,v_Vel,v_Acc,Preceding,Space_Headway,Time_Headway,Location,Relative_Time,Preceding_Vehicle_Class,Rear_to_Front_Space_Headway,Follower_Minus_Preceding_Velocity_diff,v_Class_Name
9365776,572,1113437894100,17.928,1485.487,6042644.407,2134541.664,4.35864,2,0.0,0.0,566,0.0,9999.99,i-80,1112274875001,NaN,NaN,NaN,Car


In [116]:
#filtered_vClass_U= filtered_ngsim_U[['Vehicle_ID','v_Class','v_length']].drop_duplicates().sort_values('v_Class')
filtered_vClass_U= filtered_ngsim_U.groupby(['Vehicle_ID','v_Class'], as_index=False)['v_length'].mean().sort_values('v_Class')
v_Class_M_U     =set(filtered_vClass_U[(filtered_vClass_U['v_Class'] == 1 )  ]['Vehicle_ID'])
v_Class_C_U     =set(filtered_vClass_U[(filtered_vClass_U['v_Class'] == 2 )  ]['Vehicle_ID'])
v_Class_HV_U    =set(filtered_vClass_U[(filtered_vClass_U['v_Class'] == 3 )  ]['Vehicle_ID'])

#df.groupby(["state", "gender"])["last_name"].count()

#filtered_vClass_I= filtered_ngsim_I[['Vehicle_ID','v_Class','v_length']].drop_duplicates().sort_values('v_Class')
filtered_vClass_I = filtered_ngsim_I.groupby(['Vehicle_ID','v_Class'], as_index=False)['v_length'].mean().sort_values('v_Class')
v_Class_M_I     =set(filtered_vClass_I[(filtered_vClass_I['v_Class'] == 1 )  ]['Vehicle_ID'])
v_Class_C_I     =set(filtered_vClass_I[(filtered_vClass_I['v_Class'] == 2 )  ]['Vehicle_ID'])
v_Class_HV_I    =set(filtered_vClass_I[(filtered_vClass_I['v_Class'] == 3 )  ]['Vehicle_ID'])

KeyError: 'Vehicle_ID'

In [142]:
filtered_ngsim_U.index

MultiIndex([(      0, 515, 500, 1117685055901),
            (      7, 744, 740, 1117685162101),
            (     15, 496, 510, 1117685099501),
            (     17, 946, 936, 1117685233301),
            (     28,   5,   0, 1117684841001),
            (     36, 686, 680, 1117685181701),
            (     39, 709,   0, 1117685215701),
            (     43, 485, 479, 1117686004401),
            (     44, 182, 170, 1117684040101),
            (     48, 143, 135, 1117684939401),
            ...
            (8704767, 406, 401, 1117685912901),
            (8704772, 711, 704, 1117685139001),
            (8704774, 447, 442, 1117686020201),
            (8704777, 375, 384, 1117685985801),
            (8704781, 427, 422, 1117684064001),
            (8704785, 582, 561, 1117684102401),
            (8704787, 690, 680, 1117684133501),
            (8704790, 347, 338, 1117685038201),
            (8704793, 448, 445, 1117685991101),
            (8704797, 601, 593, 1117684146901)],
           names=[None,

In [145]:

filtered_ngsim_U.index.rename(['id', 'Vehicle_ID', 'Preceding', 'Relative_Time'], inplace=True)

In [140]:
filtered_ngsim_U.xs((500,515),level=("Preceding",'Vehicle_ID'))

,,Global_Time,Local_X,Local_Y,Global_X,Global_Y,v_length,v_Class,v_Vel,v_Acc,Space_Headway,Time_Headway,Location,Preceding_Vehicle_Class,Rear_to_Front_Space_Headway,Follower_Minus_Preceding_Velocity_diff,v_Class_Name
,Relative_Time,,,,,,,,,,,,,,,,
0,1117685055901,1118848075000,30.034,188.062,6451203.729,1873252.549,3.9624,2,7.104888,0.624840,36.301680,5.11,us-101,NaN,NaN,NaN,Car
1098,1117685056001,1118848075100,30.063,190.410,6451205.318,1873250.798,3.9624,2,7.150608,0.277368,36.463224,5.10,us-101,NaN,NaN,NaN,Car
9508,1117685060201,1118848079300,28.549,281.971,6451270.837,1873184.648,3.9624,2,6.065520,-1.286256,44.772072,7.38,us-101,NaN,NaN,NaN,Car
42453,1117685105001,1118848124100,29.607,1082.700,6451866.567,1872647.592,3.9624,2,4.303776,3.413760,29.303472,6.81,us-101,NaN,NaN,NaN,Car
52236,1117685099401,1118848118500,29.868,1015.798,6451815.977,1872691.287,3.9624,2,3.526536,1.228344,26.956512,7.64,us-101,NaN,NaN,NaN,Car
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8665083,1117685078501,1118848097600,28.976,782.361,6451640.883,1872845.922,3.9624,2,8.842248,2.368296,38.770560,4.38,us-101,NaN,NaN,NaN,Car
8669638,1117685071601,1118848090700,30.432,576.420,6451485.551,1872981.540,3.9624,2,9.238488,0.835152,55.604664,6.02,us-101,NaN,NaN,NaN,Car
8672041,1117685104701,1118848123800,29.649,1078.958,6451863.748,1872649.992,3.9624,2,3.605784,0.566928,29.004768,8.04,us-101,NaN,NaN,NaN,Car


In [160]:
filtered_ngsim_U['id'][filtered_ngsim_U.xs((0),level=("id"))]

KeyError: 'id'

In [137]:
filtered_ngsim_U.index

MultiIndex([(      0, 515, 500, 1117685055901),
            (      7, 744, 740, 1117685162101),
            (     15, 496, 510, 1117685099501),
            (     17, 946, 936, 1117685233301),
            (     28,   5,   0, 1117684841001),
            (     36, 686, 680, 1117685181701),
            (     39, 709,   0, 1117685215701),
            (     43, 485, 479, 1117686004401),
            (     44, 182, 170, 1117684040101),
            (     48, 143, 135, 1117684939401),
            ...
            (8704767, 406, 401, 1117685912901),
            (8704772, 711, 704, 1117685139001),
            (8704774, 447, 442, 1117686020201),
            (8704777, 375, 384, 1117685985801),
            (8704781, 427, 422, 1117684064001),
            (8704785, 582, 561, 1117684102401),
            (8704787, 690, 680, 1117684133501),
            (8704790, 347, 338, 1117685038201),
            (8704793, 448, 445, 1117685991101),
            (8704797, 601, 593, 1117684146901)],
           names=[None,

In [156]:
#filtered_ngsim_U.index.get_level_values("") == 
filtered_ngsim_U['v_Vel'][filtered_ngsim_U.index.get_level_values("Vehicle_ID") == 500 ]

id       Vehicle_ID  Preceding  Relative_Time
10727    500         495        1117685085401     1.524000
16529    500         488        1117684117901    17.416272
17344    500         488        1117684112301    17.644872
21393    500         495        1117685073301     7.620000
31466    500         0          1117686016301    16.754856
                                                   ...    
8676583  500         495        1117685141501     4.367784
8683708  500         0          1117686046501     4.590288
8686647  500         495        1117685143701     5.370576
8695071  500         0          1117685151301     6.553200
8702978  500         494        1117686028001    14.249400
Name: v_Vel, Length: 1923, dtype: float64

In [118]:
result = []
velocity_diff=[]
space_headway_rear_to_front = []

for i in filtered_ngsim_U.index.get_level_values(0):
    print(i)
    if i%500000==0:
        print(f"row count{millify(i, precision=2, drop_nulls=False)} processed")
    #this row below is time taking

    #velocity = filtered_ngsim_U['v_Vel'][( filtered_ngsim_U['Vehicle_ID']== filtered_ngsim_U['Preceding'][i] ) & (filtered_ngsim_U['Relative_Time']== filtered_ngsim_U['Relative_Time'][i]) ] - filtered_ngsim_U['v_Vel'][i]
    velocity = filtered_ngsim_U['v_Vel'][( filtered_ngsim_U.index.get_level_values("Vehicle_ID")== filtered_ngsim_U.index.get_level_values(i,"Preceding") ) & (filtered_ngsim_U.index.get_level_values("Relative_Time")== filtered_ngsim_U.index.get_level_values(i,"Relative_Time") ) ] - filtered_ngsim_U['v_Vel'][i]
    #this row above is time taking
    #space = filtered_ngsim_U['Space_Headway'][i]  - filtered_vClass_U['v_length'][filtered_vClass_U['Vehicle_ID']== filtered_ngsim_U['Preceding'][i]]
     #print(f"i:{i,} space: {space}, filtered_ngsim_U['Space_Headway'][i]: {filtered_ngsim_U['Space_Headway'][i]},relative time={filtered_ngsim_U['Relative_Time'][i]},preceding vehicle:{filtered_ngsim_U['Preceding'][i]}   minus :{filtered_vClass_U['v_length'][filtered_vClass_U['Vehicle_ID']== filtered_ngsim_U['Preceding'][i]].mean()} ")
    space_headway_rear_to_front.append(space)
    velocity_diff.append(velocity)
    
    if filtered_ngsim_U['Preceding'][i] in v_Class_C_U:
        result.append('Car')
    elif filtered_ngsim_U['Preceding'][i] in v_Class_HV_U:
        result.append('Heavy Vehicle')
    elif filtered_ngsim_U['Preceding'][i] in v_Class_M_U:
        result.append('Motorcycle')
    else: 
        result.append(np.NaN)

filtered_ngsim_U['Preceding_Vehicle_Class'] = result
filtered_ngsim_U['Follower_Minus_Preceding_Velocity_diff'] = velocity_diff
filtered_ngsim_U['Rear_to_Front_Space_Headway'] = space_headway_rear_to_front


0
row count0.00 processed


TypeError: get_level_values() takes 2 positional arguments but 3 were given

In [ ]:
'''rEMOVE THIS AFTER VALIDATING THE US-101 FILE 
result = []
space_headway_rear_to_front = []

for i in filtered_ngsim_I.index:
    if filtered_ngsim_I['Preceding'][i] in v_Class_C_I:
        result.append('Car')
    elif filtered_ngsim_I['Preceding'][i] in v_Class_HV_I:
        result.append('Heavy Vehicle')
    elif filtered_ngsim_I['Preceding'][i] in v_Class_M_I:
        result.append('Motorcycle')
    else: 
        result.append(np.NaN)
    velocity = filtered_ngsim_I['v_Vel'][( filtered_ngsim_I['Vehicle_ID']== filtered_ngsim_I['Preceding'][i] ) & (filtered_ngsim_I['Relative_Time']== filtered_ngsim_I['Relative_Time'][i]) ] - filtered_ngsim_I['v_Vel'][i]
    space=filtered_ngsim_I['Space_Headway'][i]  - filtered_vClass_I['v_length'][filtered_vClass_I['Vehicle_ID']== filtered_ngsim_I['Preceding'][i]]
    velocity_diff.append(velocity)
    space_headway_rear_to_front.append(space)
    if i%500000==0:
        print(f"row count{millify(i, precision=2, drop_nulls=False)} processed")

filtered_ngsim_I['Rear_to_Front_Space_Headway'] = space_headway_rear_to_front
filtered_ngsim_I['Follower_Minus_Preceding_Velocity_diff'] = velocity_diff
filtered_ngsim_I['Preceding_Vehicle_Class'] = result
'''

In [ ]:
'''
space_headway_rear_to_front = []

for i in filtered_ngsim_I.index:
    space=filtered_ngsim_I['Space_Headway'][i]  - filtered_vClass_I['v_length'][filtered_vClass_I['Vehicle_ID']== filtered_ngsim_I['Preceding'][i]].mean()
    space_headway_rear_to_front.append(space)
    if i%500000==0:
        print(f"row count{millify(i, precision=2, drop_nulls=False)} processed")
filtered_ngsim_I['Rear_to_Front_Space_Headway'] = space_headway_rear_to_front
'''

KeyboardInterrupt: 

In [ ]:
'''
space_headway_rear_to_front = []

for i in filtered_ngsim_U.index:
    space = filtered_ngsim_U['Space_Headway'][i]  - filtered_vClass_U['v_length'][filtered_vClass_U['Vehicle_ID']== filtered_ngsim_U['Preceding'][i]].mean()
    space_headway_rear_to_front.append(space)
    if i%500000==0:
        print(f"row count{millify(i, precision=2, drop_nulls=False)} processed")
        

filtered_ngsim_U['Rear_to_Front_Space_Headway'] = space_headway_rear_to_front
'''

row count0.00 processed
row count500.00k processed
row count2.50M processed
row count3.50M processed
row count4.00M processed
row count5.00M processed
row count5.50M processed
row count6.00M processed
row count6.50M processed
row count7.50M processed
row count8.00M processed
row count8.50M processed


In [ ]:
filtered_ngsim_U['Vehicle_combination']= filtered_ngsim_U['Preceding_Vehicle_Class'] + '-' + filtered_ngsim_U['v_Class_Name']
filtered_ngsim_I['Vehicle_combination']= filtered_ngsim_I['Preceding_Vehicle_Class'] + '-' + filtered_ngsim_I['v_Class_Name']

In [ ]:
filtered_ngsim= pd.concat([filtered_ngsim_U , filtered_ngsim_I])

In [ ]:
print(filtered_ngsim_U.shape[0])
print(filtered_ngsim_I.shape[0])
print(filtered_ngsim.shape[0])

4098933
4555939
8654872


In [ ]:
stringpath= str(p)[0:str(p).rfind('\\')] + '\\data'
ngsimfilteredfile=stringpath + '\\' + 'Cleaned_NGSIM_Data_Part_1.csv'
filtered_ngsim.to_csv(ngsimfilteredfile, index = False)

In [ ]:
#this is converted to feet data
filtered_vClass_U['v_length'][filtered_vClass_U['Vehicle_ID']== 500 ].mean()

5.384799999999999

In [ ]:
filtered_ngsim_U[:2]

,Vehicle_ID,Global_Time,Local_X,Local_Y,Global_X,Global_Y,v_length,v_Class,v_Vel,v_Acc,Preceding,Space_Headway,Time_Headway,Location,Relative_Time,Preceding_Vehicle_Class,Rear_to_Front_Space_Headway,Follower_Minus_Preceding_Velocity_diff,v_Class_Name
0,515,1118848075000,30.034,188.062,6451203.729,1873252.549,3.9624,2,7.104888,0.62484,500,36.301680,5.11,us-101,1117685055901,NaN,NaN,NaN,Car
3,2127,1118847624800,19.632,1775.614,6452425.122,1872172.475,4.1148,2,11.436096,3.41376,2124,14.910816,1.30,us-101,1117684605701,NaN,NaN,NaN,Car


In [ ]:



print(filtered_ngsim_U['v_Vel'][((filtered_ngsim_U['Vehicle_ID'] == 500 ) & (filtered_ngsim_U['Relative_Time'] == 1117685055901 )) ])

print(filtered_ngsim_U['v_Vel'][((filtered_ngsim_U['Vehicle_ID'] == 515 ) & (filtered_ngsim_U['Relative_Time'] == 1117685055901 )) ])

7061760    8.772144
Name: v_Vel, dtype: float64
0    7.104888
Name: v_Vel, dtype: float64


In [ ]:
filtered_ngsim_U['Relative_Time'].dtype

dtype('int64')

KeyError: "None of ['Vehicle_ID', 'Relative_Time'] are in the columns"

In [ ]:
filtered_ngsim_U


,,,,Global_Time,Local_X,Local_Y,Global_X,Global_Y,v_length,v_Class,v_Vel,v_Acc,Preceding,Space_Headway,Time_Headway,Location,Rear_to_Front_Space_Headway,Follower_Minus_Preceding_Velocity_diff,v_Class_Name
,Vehicle_ID,Preceding_Vehicle_Class,Relative_Time,,,,,,,,,,,,,,,,
0,515,NaN,1117685055901,1118848075000,30.034,188.062,6451203.729,1873252.549,3.96240,2,7.104888,0.624840,500,36.301680,5.11,us-101,NaN,NaN,Car
7,744,NaN,1117685162101,1118848181200,28.878,490.086,6451422.353,1873041.018,4.57200,2,7.479792,-0.012192,740,11.521440,1.54,us-101,NaN,NaN,Car
15,496,NaN,1117685099501,1118848118600,52.141,1656.454,6452284.002,1872253.037,4.57200,2,9.933432,-2.112264,510,20.790408,2.09,us-101,NaN,NaN,Car
17,946,NaN,1117685233301,1118848252400,5.373,365.819,6451347.228,1873143.047,3.96240,2,6.909816,0.000000,936,18.574512,2.69,us-101,NaN,NaN,Car
28,5,NaN,1117684841001,1118847860100,4.948,149.178,6451195.587,1873298.132,3.96240,2,0.000000,0.000000,0,0.000000,0.00,us-101,NaN,NaN,Car
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8704785,582,NaN,1117684102401,1118847121500,27.986,210.580,6451248.662,1873208.150,23.19528,3,10.229088,3.413760,561,31.476696,3.08,us-101,NaN,NaN,Heavy Vehicle
8704787,690,NaN,1117684133501,1118847152600,40.584,298.509,6451303.292,1873136.542,5.02920,2,11.780520,1.036320,680,27.535632,2.34,us-101,NaN,NaN,Car
8704790,347,NaN,1117685038201,1118848057300,41.208,1802.654,6452401.087,1872164.685,4.41960,2,7.620000,0.000000,338,25.383744,3.33,us-101,NaN,NaN,Car


Int64Index([515, 744, 496, 946,   5, 686, 709, 485, 182, 143,
            ...
            406, 711, 447, 375, 427, 582, 690, 347, 448, 601],
           dtype='int64', name='Vehicle_ID', length=1645876)

In [147]:
filtered_ngsim_U[((filtered_ngsim_U['Space_Headway'] == 0 ) & (filtered_ngsim_U.index.get_level_values("Preceding") > 0 )) ]
#filtered_ngsim_U[((filtered_ngsim_U['Space_Headway'] == 0 ) & (filtered_ngsim_U['Preceding'] > 0 )) ]

,,,,Global_Time,Local_X,Local_Y,Global_X,Global_Y,v_length,v_Class,v_Vel,v_Acc,Space_Headway,Time_Headway,Location,Preceding_Vehicle_Class,Rear_to_Front_Space_Headway,Follower_Minus_Preceding_Velocity_diff,v_Class_Name
id,Vehicle_ID,Preceding,Relative_Time,,,,,,,,,,,,,,,,
76361,99,92,1117684929401,1118847948500,12.942,1420.624,6452132.084,1872437.889,2.5908,1,17.373600,-0.064008,0.0,0.0,us-101,NaN,NaN,NaN,Motorcycle
107058,806,814,1117685210701,1118848229800,6.699,796.216,6451666.005,1872853.511,4.4196,2,2.795016,0.000000,0.0,0.0,us-101,NaN,NaN,NaN,Car
452347,366,370,1117685952801,1118848971900,12.613,835.132,6451695.487,1872819.732,2.1336,1,8.695944,1.816608,0.0,0.0,us-101,NaN,NaN,NaN,Motorcycle


In [119]:
filtered_ngsim_U.index.get_level_values("Preceding")

Int64Index([500, 740, 510, 936,   0, 680,   0, 479, 170, 135,
            ...
            401, 704, 442, 384, 422, 561, 680, 338, 445, 593],
           dtype='int64', name='Preceding', length=1645876)

In [226]:
filtered_ngsim_U.xs((filtered_ngsim_U[(filtered_ngsim_U.index.get_level_values("id") == 0 )].index.get_level_values("Relative_Time")[0],filtered_ngsim_U[(filtered_ngsim_U.index.get_level_values("id") == 0 )].index.get_level_values("Preceding").values[0]),level=("Relative_Time",'Vehicle_ID')).index.get_level_values('id').values[0] 

7061760

In [229]:
velocity_diff=[]

for i in filtered_ngsim_U.index.get_level_values(0):
    #print(i)
    if i%100==0:
        print(f"row count{millify(i, precision=2, drop_nulls=False)} processed")
    preceding_corresponding_id=filtered_ngsim_U.xs((filtered_ngsim_U[(filtered_ngsim_U.index.get_level_values("id") == 0 )].index.get_level_values("Relative_Time")[0],filtered_ngsim_U[(filtered_ngsim_U.index.get_level_values("id") == 0 )].index.get_level_values("Preceding").values[0]),level=("Relative_Time",'Vehicle_ID')).index.get_level_values('id').values[0] 
    preceding_corresponding_id
    #print(f"velocity previous :{ filtered_ngsim_U['v_Vel'][(filtered_ngsim_U.index.get_level_values('id') == preceding_corresponding_id ) ].values}")
    #print(f"velocity current : { filtered_ngsim_U['v_Vel'][(filtered_ngsim_U.index.get_level_values('id') == i ) ].values}")
    velocity = filtered_ngsim_U['v_Vel'][(filtered_ngsim_U.index.get_level_values('id') == i ) ].values - filtered_ngsim_U['v_Vel'][(filtered_ngsim_U.index.get_level_values('id') == preceding_corresponding_id ) ].values
    #if i >= 0:
    #    break    
    velocity_diff.append(velocity)

row count0.00 processed


In [165]:
filtered_ngsim_U.xs((0,500,515),level=("id","Preceding",'Vehicle_ID'),"v_Vel")

SyntaxError: positional argument follows keyword argument (2058454446.py, line 1)

In [169]:
filtered_ngsim_U["v_Class"][(filtered_ngsim_U.index.get_level_values("Vehicle_ID") == 515 ) & (filtered_ngsim_U.index.get_level_values("Preceding") == 500 ) &(filtered_ngsim_U.index.get_level_values("id") == 0 ) ]

id  Vehicle_ID  Preceding  Relative_Time
0   515         500        1117685055901    2
Name: v_Class, dtype: int64

In [170]:
filtered_ngsim_U["v_Class"][(filtered_ngsim_U.index.get_level_values("Vehicle_ID") == 515 ) & (filtered_ngsim_U.index.get_level_values("Preceding") == 500 ) &(filtered_ngsim_U.index.get_level_values("id") == 0 ) ].index

MultiIndex([(0, 515, 500, 1117685055901)],
           names=['id', 'Vehicle_ID', 'Preceding', 'Relative_Time'])

In [177]:
filtered_ngsim_U[(filtered_ngsim_U.index.get_level_values("Vehicle_ID") == 515 ) & (filtered_ngsim_U.index.get_level_values("Preceding") == 500 ) &(filtered_ngsim_U.index.get_level_values("id") == 0 ) ].index.get_level_values("Relative_Time")

Int64Index([0], dtype='int64', name='id')

In [178]:
filtered_ngsim_U[(filtered_ngsim_U.index.get_level_values("id") == 0 )].index.get_level_values("Relative_Time")

Int64Index([1117685055901], dtype='int64', name='Relative_Time')

In [206]:
filtered_ngsim_U[(filtered_ngsim_U.index.get_level_values("id") == 0 )].index.get_level_values("Preceding").values[0]


500

In [211]:
filtered_ngsim_U["v_Vel"][(filtered_ngsim_U.index.get_level_values("id") == 515 ) ]

id   Vehicle_ID  Preceding  Relative_Time
515  21          8          1117683986301    13.267944
Name: v_Vel, dtype: float64